In [4]:
import gradio as gr
import pandas as pd
from transformers import TapasTokenizer, TapasForQuestionAnswering
from transformers import AutoTokenizer, AutoModelForTableQuestionAnswering
from transformers import pipeline

c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# df = pd.read_excel(r"C:\Users\michi\enterpriseAI_michi\enterprise-ai-project\module_guide_tableQA\03_extracted_final_modules\BA_IS_all_modules.xlsx")

# df = df.head(15)
# df = df.astype(str)

# # remove all leading and tailing whitespaces
# df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
# df.to_excel(r"C:\Users\michi\enterpriseAI_michi\enterprise-ai-project\module_guide_tableQA\03_extracted_final_modules\BA_IS_all_modules_15.xlsx")

In [13]:
def get_answer(dropdown, question, view_as_table=False, model="google/tapas-finetuned-wtq"):
    df = pd.DataFrame()
    if dropdown == "Master Information Systems":
        df = pd.read_excel(
            r"03_extracted_final_modules\MS_IS_all_modules_orginal_15_rows_cleaned.xlsx"
        )
    elif dropdown == "Bachelor Information Systems":
        df = pd.read_excel(r"03_extracted_final_modules\BA_IS_all_modules_15.xlsx")
    elif dropdown == "Bachelor Management":
        df = pd.read_excel(r"03_extracted_final_modules\BA_MM_all_modules_15.xlsx")
    df = df.astype(str)
    print(question)
    question = str(question)
    print(df.shape)
    question = [question]
                
    if (model == "google/tapas-finetuned-wtq"):
        tqa = pipeline(
            task="table-question-answering", model="google/tapas-base-finetuned-wtq"
        )
    elif (model == "microsoft/tapex-finetuned-wtq"):
        tqa = pipeline(task="table-question-answering", model="microsoft/tapex-base-finetuned-wtq")

    results = tqa(table=df, query=question)
    print(results)
    cells_input = results["cells"]
    cells_input = str(cells_input)
    cells_input = cells_input.replace("[", "")
    cells_input = cells_input.replace("]", "")
    cells_input = cells_input.replace("'", "")

    print(cells_input)
    print(results)
    html_string_short = f"<h1>Short Answer:</h1><p>{cells_input}</p>"
    row_numbers = [coord[0] for coord in results["coordinates"]]
    df_short = df.iloc[row_numbers]
    df_short = df_short.dropna(axis=1, how="all")
    df_short = df_short.loc[:, (df_short != "--").any(axis=0)]
    html_table = f"<hr><h2>Complete Module(s):</h2><p>{df_short.to_html(index=False)}</p>"

    # check if there are more than 1 rows in df_short
    html_string = ""
    if df_short.shape[0] > 1 or view_as_table == True:
        html_string = html_table
    else:
        html_string = """<html>
    <head>
        <style>
            .module-info {
                margin-bottom: 20px;
            }
            .module-info h3, .module-info p {
                margin: 0;
            }
        </style>
    </head>
    <body>
        <hr>
        <h1>Detailed Module Information</h1>

        <div class="module-info">
            <h3>Module title:</h3>
            <p>Project Seminar</p>
        </div>
        <div class="module-info">
            <h3>Abbreviation:</h3>
            <p>12-PS-192-m01</p>
        </div>
        <div class="module-info">
            <h3>Module coordinator:</h3>
            <p>Faculty of Business Management and Economics</p>
        </div>
        <div class="module-info">
            <h3>Module offered by:</h3>
            <p>Holder of the Chair of Business Management and Business</p>
        </div>
        <div class="module-info">
            <h3>ETCS:</h3>
            <p>15</p>
        </div>
        <div class="module-info">
            <h3>Method of grading:</h3>
            <p>numerical grade</p>
        </div>
        <div class="module-info">
            <h3>Duration:</h3>
            <p>1 semester</p>
        </div>
        <div class="module-info">
            <h3>Module level:</h3>
            <p>graduate</p>
        </div>
        <div class="module-info">
            <h3>Contents:</h3>
            <p>In small project teams of 4 to 10 members, students will spend several months actively working on a specific and realistic problem with practical relevance. They will progress through several project stages including as-is analysis, to-be conception and implementation of an IS solution. The project teams will be required to work independently and will only receive advice and minor support from research assistants.</p>
        </div>
        <div class="module-info">
            <h3>Intended learning outcomes:</h3>
            <ul>
                <li>Analyze business tasks and requirements and generate fitting IS solutions</li>
                <li>Apply project management methods</li>
                <li>Internalize stress, time and conflict management by means of practical teamwork</li>
            </ul>
        </div>
        <div class="module-info">
            <h3>Courses:</h3>
            <p>Project: preparing a conceptual design (approx. 150 hours), designing and implementing an approach to solution (approx. 300 hours) as well as presentation (approx. 20 minutes), weighted 1:2:1</p>
            <p>Language of assessment: German, English</p>
            <p>Creditable for bonus</p>
        </div>
        <div class="module-info">
            <h3>Workload:</h3>
            <p>450 hours</p>
        </div>
    </body>
    </html>"""

    return html_string_short, html_string


with gr.Blocks() as demo:
    gr.HTML(
        "<h1>Your Module Guide Assistant</h1><p>Select a study subject you want to know more about the module guide.</p>"
    )
    table = gr.Dropdown(
        [
            "Master Information Systems",
            "Bachelor Information Systems",
            "Bachelor Management",
        ],
        label="Select Module Guide",
        value="Master Information Systems",
    )

    gr.HTML(
        "<p>Ask whatever you want to know about the module guide here. You can ask formality based and content based questions. </p>"
    )
    question = gr.Textbox(
        label="Question", value="How many ECTS credits does the project seminar have?"
    )
    with gr.Accordion("Advanced Options"):
       with gr.Group():
            model_selction = gr.Dropdown(
            [   
                "google/tapas-finetuned-wtq",
                "microsoft/tapex-finetuned-wtq",
            ], label="Select Model",
            value="google/tapas-finetuned-wtq")
            view_as_table_or_text = gr.Checkbox(label="View detailed information as table", value=False)
           
        
    ask_btn = gr.Button("Ask The Assistant")
    gr.HTML("<hr>")
    inputs = [table, question, view_as_table_or_text, model_selction]
   
    output_question = gr.HTML(label="Answer")
    outout_full_module = gr.HTML(label="Detailed Description")
    outputs = [output_question, outout_full_module]
    ask_btn.click(fn=get_answer, inputs=inputs, outputs=outputs, api_name="greet")

demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
